In [4]:
from bs4 import BeautifulSoup
import requests
url = 'https://jobsinmaine.com/jobs'
page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

print(soup)

<!DOCTYPE html>
<html lang="en">
<head>
<!-- Required meta tags -->
<meta charset="utf-8"/>
<title>All Jobs in Maine - JobsInMaine.com: Jobs in Maine</title>
<meta content="All Jobs in Maine" name="description"/>
<link href="https://jobsinmaine.com/jobs" rel="canonical"/>
<link href="/jobs?page=2" rel="next"/>
<meta content="All Jobs in Maine" property="og:title"/>
<meta content="All Jobs in Maine" property="og:description"/>
<meta content="https://jobsinmaine.com/jobs" property="og:url"/>
<meta content="website" property="og:type"/>
<meta content="https://d3535lqr6sqxto.cloudfront.net/cache/cover_images/30a8058c3d32a7790e6d5a027f1a57d208f3d6cb-2c66d8f1de28c78a776922391a6168fe5ce29ad3.jpg" property="og:image"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="https://d3535lqr6sqxto.cloudfront.net/cache/cover_images/30a8058c3d32a7790e6d5a027f1a57d208f3d6cb-2c66d8f1de28c78a776922391a6168fe5ce29ad3.jpg" name="twitter:image"/>
<meta content="All Jobs in Maine" name=

In [7]:
soup.find_all('div')

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!pip install requests beautifulsoup4



In [9]:
import requests
from bs4 import BeautifulSoup
import csv
import json
from datetime import datetime
import time

class JobsInMaineScraper:
    def __init__(self):
        self.base_url = "https://jobsinmaine.com"
        self.jobs_url = f"{self.base_url}/jobs"
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

    def scrape_jobs(self, max_pages=1):
        """
        Scrape job listings from JobsInMaine.com

        Args:
            max_pages: Number of pages to scrape (default: 1)

        Returns:
            List of job dictionaries
        """
        all_jobs = []

        for page in range(1, max_pages + 1):
            print(f"Scraping page {page}...")

            # Construct URL with page parameter
            url = f"{self.jobs_url}?page={page}" if page > 1 else self.jobs_url

            try:
                response = requests.get(url, headers=self.headers)
                response.raise_for_status()

                soup = BeautifulSoup(response.content, 'html.parser')

                # Find all job listings
                # Based on the structure, jobs appear to be in a specific format
                job_elements = soup.find_all('a', href=lambda x: x and '/jobs/' in x and '/companies/' not in x)

                # Extract job details from the page
                jobs_on_page = self.extract_job_details(soup)
                all_jobs.extend(jobs_on_page)

                print(f"Found {len(jobs_on_page)} jobs on page {page}")

                # Be respectful - add a small delay between requests
                if page < max_pages:
                    time.sleep(2)

            except requests.exceptions.RequestException as e:
                print(f"Error scraping page {page}: {e}")
                break

        return all_jobs

    def extract_job_details(self, soup):
        """
        Extract job details from the page soup

        Args:
            soup: BeautifulSoup object of the page

        Returns:
            List of job dictionaries
        """
        jobs = []

        # Find job containers - they appear to have company logos and info
        # Looking for patterns in the HTML structure
        page_text = soup.get_text()

        # Try to find job links
        job_links = soup.find_all('a', href=lambda x: x and x.startswith('/jobs/') and not x.startswith('/jobs/in-'))

        for link in job_links:
            try:
                job = {}
                job['url'] = self.base_url + link['href']

                # Find parent container to get related information
                parent = link.find_parent()
                while parent and parent.name != 'body':
                    text = parent.get_text(strip=True)
                    if text:
                        # Look for salary information
                        if '$' in text and ('/' in text or 'year' in text or 'hour' in text):
                            if 'salary' not in job:
                                job['salary'] = self.extract_salary(text)

                        # Look for time posted
                        if any(time_str in text for time_str in ['ago', 'd ago', 'w ago', 'h ago']):
                            if 'posted' not in job:
                                job['posted'] = self.extract_time_posted(text)

                    parent = parent.find_parent()

                if job['url'] not in [j.get('url') for j in jobs]:
                    jobs.append(job)

            except Exception as e:
                print(f"Error extracting job: {e}")
                continue

        # Alternative approach: Parse the visible text structure
        # This is a backup method based on the text format observed
        companies = soup.find_all('a', href=lambda x: x and '/companies/' in x)

        for i, company in enumerate(companies):
            try:
                job = {}

                # Get company name
                company_name = company.get_text(strip=True)
                if company_name:
                    job['company'] = company_name
                    job['company_url'] = self.base_url + company['href']

                # Get location (usually appears after company)
                next_sibling = company.find_next_sibling()
                if next_sibling and next_sibling.name == 'a' and '/jobs/in-' in next_sibling.get('href', ''):
                    job['location'] = next_sibling.get_text(strip=True)

                # Look for salary and time in surrounding text
                parent_text = company.find_parent().get_text() if company.find_parent() else ""

                # Extract salary
                if '$' in parent_text:
                    job['salary'] = self.extract_salary(parent_text)

                # Extract posting time
                if 'ago' in parent_text:
                    job['posted'] = self.extract_time_posted(parent_text)

                if len(job) > 1:  # Only add if we got more than just basic info
                    jobs.append(job)

            except Exception as e:
                print(f"Error in alternative extraction: {e}")
                continue

        return jobs

    def extract_salary(self, text):
        """Extract salary information from text"""
        import re
        salary_pattern = r'\$[\d,]+k?\s*-?\s*\$?[\d,]+k?\s*/\s*(?:year|hour)'
        match = re.search(salary_pattern, text)
        return match.group(0) if match else None

    def extract_time_posted(self, text):
        """Extract time posted from text"""
        import re
        time_pattern = r'\d+[dhw]\s+ago|today|yesterday'
        match = re.search(time_pattern, text)
        return match.group(0) if match else None

    def save_to_csv(self, jobs, filename='jobs_in_maine.csv'):
        """Save jobs to CSV file"""
        if not jobs:
            print("No jobs to save")
            return

        keys = set()
        for job in jobs:
            keys.update(job.keys())

        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=sorted(keys))
            writer.writeheader()
            writer.writerows(jobs)

        print(f"Saved {len(jobs)} jobs to {filename}")

    def save_to_json(self, jobs, filename='jobs_in_maine.json'):
        """Save jobs to JSON file"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(jobs, f, indent=2, ensure_ascii=False)

        print(f"Saved {len(jobs)} jobs to {filename}")

# Example usage
if __name__ == "__main__":
    scraper = JobsInMaineScraper()

    # Scrape jobs from the first 3 pages
    jobs = scraper.scrape_jobs(max_pages=3)

    print(f"\nTotal jobs scraped: {len(jobs)}")

    # Save to CSV
    scraper.save_to_csv(jobs)

    # Save to JSON
    scraper.save_to_json(jobs)

    # Print first few jobs as preview
    print("\nFirst 5 jobs:")
    for i, job in enumerate(jobs[:5], 1):
        print(f"\n{i}. {job}")

Scraping page 1...
Found 108 jobs on page 1
Scraping page 2...
Found 92 jobs on page 2
Scraping page 3...
Found 90 jobs on page 3

Total jobs scraped: 290
Saved 290 jobs to jobs_in_maine.csv
Saved 290 jobs to jobs_in_maine.json

First 5 jobs:

1. {'url': 'https://jobsinmaine.com/jobs/153982321-merchandiser', 'posted': '13h ago', 'salary': '$65k - $68k / year'}

2. {'url': 'https://jobsinmaine.com/jobs/153982321/apply', 'posted': '13h ago', 'salary': '$65k - $68k / year'}

3. {'url': 'https://jobsinmaine.com/jobs/167606428-postdoctoral-associate', 'posted': '14h ago', 'salary': '$65k - $68k / year'}

4. {'url': 'https://jobsinmaine.com/jobs/167606428/apply', 'posted': '14h ago', 'salary': '$65k - $68k / year'}

5. {'url': 'https://jobsinmaine.com/jobs/168636212-customer-relationship-management-platform-manager', 'salary': '$65k - $68k / year', 'posted': '3d ago'}


In [10]:
scraper = JobsInMaineScraper()
jobs = scraper.scrape_jobs(max_pages=1)

# Scrape multiple pages
jobs = scraper.scrape_jobs(max_pages=5)

# Save results
scraper.save_to_csv(jobs, 'my_jobs.csv')
scraper.save_to_json(jobs, 'my_jobs.json')

Scraping page 1...
Found 108 jobs on page 1
Scraping page 1...
Found 108 jobs on page 1
Scraping page 2...
Found 92 jobs on page 2
Scraping page 3...
Found 90 jobs on page 3
Scraping page 4...
Found 90 jobs on page 4
Scraping page 5...
Found 90 jobs on page 5
Saved 470 jobs to my_jobs.csv
Saved 470 jobs to my_jobs.json
